In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from colorama import  Back, Style, Fore


In [37]:
print(Style.BRIGHT + Fore.CYAN + "This is a simple web scraper that scrapes the top 100 movies of all time from the IMDB website")

This is a simple web scraper that scrapes the top 100 movies of all time from the IMDB website


## Trying and testing level

In [8]:
recuest = requests.get("https://wuzzuf.net/search/jobs/?a=hpb%7Cspbg&q=data%20science&start=0")
content = recuest.content
soup = BeautifulSoup(content, "html.parser")

In [50]:
jobs = soup.find_all("div", {"class": "css-1gatmva e1v1l3u10"})
for job in jobs:
    job_skills = ", " .join(job.find_all("div", {"class": "css-y4udm8"})[0].text.split(" · ")[2:])
    print(job_skills)



IT/Software Development, Engineering - Telecom/Technology, Computer Science, Algorithms, Information Technology (IT), Python, Software, Software Development, Software Engineering, Programming
Accounting/Finance, Administration, Banking, Accounting, Accounting Software, Data Entry, excel, Finance, Financial Management, Microsoft Excel
R&D/Science, Sales/Retail, Sales Field, Telesales, Data collector, Food Science
Operations/Management, Sales/Retail, Computer Science, Data Management, Computer Engineering, Data, Management, Information Technology (IT)
Analyst/Research, Engineering - Telecom/Technology, Analysis, analytical, Communication skills, Computer Science, Data, Data Analysis, Engineering, ETL
Operations/Management, Manufacturing/Production, Analyst/Research, Analysis, business, analytical, Data Analysis, Computer Science, Data, excel
IT/Software Development, Engineering - Telecom/Technology, Analysis, BI, Business Intelligence, Computer Science, Data Analysis, Data Warehousing, E

In [52]:
df_jobs_test = pd.DataFrame(columns=["Job Name", "Job Company", "Job City", "Job Country", "Job Type", "Job Level", "Job Level Years", "Job Skills"])
for job in jobs:
    job_name = job.find_all("h2", {"class": "css-m604qf"})[0].text.strip()
    job_company = job.find_all("a", {"class": "css-17s97q8"})[0].text.split("-")[0].strip()
    job_city = job.find_all("span", {"class": "css-5wys0k"})[0].text.split(",")[0].strip()
    job_country = job.find_all("span", {"class": "css-5wys0k"})[0].text.split(",")[-1].strip()
    job_type = job.find_all("span", {"class": "css-1ve4b75 eoyjyou0"})[0].text.split(",")[0].strip()
    job_level = job.find_all("div", {"class": "css-1lh32fc"})[0].next_sibling.text.split("·")[0].strip()
    job_level_years = job.find_all("div", {"class": "css-1lh32fc"})[0].next_sibling.text.split("·")[1].strip()
    job_skills = ", " .join(job.find_all("div", {"class": "css-y4udm8"})[0].text.split(" · ")[2:])

    df_jobs_test.loc[len(df_jobs_test.index)] = [job_name, job_company, job_city, job_country, job_type, job_level, job_level_years, job_skills]
df_jobs_test.head(5)


,Job Name,Job Company,Job City,Job Country,Job Type,Job Level,Job Level Years,Job Skills
0,AI & Data Science Python Developer Intern,Sequel Solutions,Cairo,Egypt,Internship,Student,0 - 1 Yrs of Exp,"IT/Software Development, Engineering - Telecom..."
1,Senior Accountant,Data Science,Hadayek October,Egypt,Full Time,Experienced,3 - 5 Yrs of Exp,"Accounting/Finance, Administration, Banking, A..."
2,Telesales Representative & Data Collector,Diamond,Haram,Egypt,Full Time,Experienced,3 - 5 Yrs of Exp,"R&D/Science, Sales/Retail, Sales Field, Telesa..."
3,Data Management Supervisor,Nahdet Misr Publishing Group,Mohandessin,Egypt,Full Time,Experienced,6 - 8 Yrs of Exp,"Operations/Management, Sales/Retail, Computer ..."
4,Data Analyst,Confidential,Mohandessin,Egypt,Full Time,Entry Level,1 - 2 Yrs of Exp,"Analyst/Research, Engineering - Telecom/Techno..."


## Extracting

In [42]:
def get_job_data(search_text):
    i = 0
    j = 0
    df_jobs = pd.DataFrame(columns=["Job Name", "Job Company", "Job City", "Job Country", "Job Type", "Job Level", "Job Level Years", "Job Skills", "Job Date"])
    urlname = search_text.replace(" ", "%20")

    while True:
        request = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb%7Cspbg&q={urlname}&start={i}")
        content = request.content
        soup = BeautifulSoup(content, "html.parser")
        jobs = soup.find_all("div", {"class": "css-1gatmva e1v1l3u10"})

        if len(jobs) == 0:
            break    
        # collect job data
        for job in jobs:
            job_name = job.find_all("h2", {"class": "css-m604qf"})[0].text.strip()
            job_company = job.find_all("a", {"class": "css-17s97q8"})[0].text.split("-")[0].strip()
            job_city = job.find_all("span", {"class": "css-5wys0k"})[0].text.split(",")[0].strip()
            job_country = job.find_all("span", {"class": "css-5wys0k"})[0].text.split(",")[-1].strip()
            job_type = job.find_all("span", {"class": "css-1ve4b75 eoyjyou0"})[0].text.split(",")[0].strip()
            job_level = job.find_all("div", {"class": "css-1lh32fc"})[0].next_sibling.text.split("·")[0].strip()
            job_level_years = job.find_all("div", {"class": "css-1lh32fc"})[0].next_sibling.text.split("·")[1].strip()
            job_skills = ", " .join(job.find_all("div", {"class": "css-y4udm8"})[0].text.split(" · ")[2:])
            try:
                job_date = job.find_all("div", {"class": "css-4c4ojb"})[0].text.strip()
            except:
                job_date = job.find_all("div", {"class": "css-do6t5g"})[0].text.strip()

            # insert job into dataframe
            df_jobs.loc[len(df_jobs.index)] = [job_name, job_company, job_city, job_country, job_type, job_level, job_level_years, job_skills, job_date]
            j += 1

        i += 1
        if i % 20 == 0:
            print(Style.BRIGHT + Fore.CYAN + "Pages Scraped:", i)
    print(Style.BRIGHT + Fore.GREEN + "Total Pages Scraped:", i, "\nTotal Jobs Scraped:", j)
    return df_jobs


In [43]:
search_text = "data science"
df_jobs = get_job_data(search_text)

Pages Scraped: 20
Pages Scraped: 40
Pages Scraped: 60
Pages Scraped: 80
Total Pages Scraped: 84 
Total Jobs Scraped: 1248


In [44]:
df_jobs.to_csv("jobs.csv")